# Coursework 3: Bilinear Inverse Problems and Low-Rank Matrix Recovery

[x] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 3.1 Test Data Generation (10%)

We consider the low-rank matrix completion problem given by 
$$
    \bm{y} = \mathcal{P}_{\Omega}(\bm{X}) 
$$
where $\bm{X} \in \mathbb{R}^{m \times n}$ is a low rank matrix of rank $r$. 

Data generation: Write $\bm{X} = \bm{U} \bm{G} \bm{V}^{\mathsf{T}}$, where $\bm{U} \in \mathbb{R}^{m \times r}$, $\bm{G} \in \mathbb{R}^{r \times r}$, and $\bm{V} \in \mathbb{R}^{n \times r}$ are matrices with i.i.d. $\mathcal{N}(0,1)$ Gaussian entries. (Note that by $\bm{X} = \bm{U} \bm{G} \bm{V}^{\mathsf{T}}$ we are not talking about SVD.)

Design and implement a function `LRMC_data_gen` to generate test data. Provide necessary documentation.

In [4]:
using Random
using LinearAlgebra
using StatsBase
using Distributions
using Plots

function GaussianGen(m::Int64, n::Int64)
    A = randn(Float64, m, n)
    Norm = zeros(m,n)
    for i = 1:n
        Norm[:,i] = normalize(A[:,i], 2);
    end
    return Norm
end

function Create_linear_operator(Ω)
    m, n = size(Ω)
    idm = Matrix(1I, m*n, m*n)

    cardinality = count(i->(i != false), vec(Ω))
    A = Array{Int64, 2}(undef, cardinality, m*n)

    idx = 1

    for i = 1:m*n
        if vec(Ω)[i] == true
            A[idx, :] = idm[i, :]
            idx += 1
        end
    end

    return A
end

function Observation_samples(X::AbstractArray, m::Int64, n::Int64, samples::Int64)

    random_indices = sample(randperm(m*n), samples, replace=false)

    Ω = zeros(Bool, m, n)
    Ω[random_indices] .= true

    Y = zeros(m, n)

    Y[Ω] = X[Ω]

    return Y, Ω
end

function LRMC_data_gen(m::Int64, n::Int64, r::Int64)
    U = GaussianGen(m, r)
    G = GaussianGen(r, r)
    V = GaussianGen(n, r)

    X = U * G * V'

    return X
end

LRMC_data_gen (generic function with 1 method)

## 3.2 Matrix Completion Techniques

In the following, the suggested simulation setup is that $m = 32$, $n=48$, $r$ varies in $2:2:8$, and $|\Omega|/mn$ varies in $\{1/8,~ 1/6,~ 1/4,~ 1/2\}$. 

### 3.2.1 Alternating Minimization (20%)

Design, implement, and run tests for the alternating minimization method for low-rank matrix completion. Use the function name `LRMCRec_AM`. Provide necessary documentation.

In [1]:
using Random
using LinearAlgebra
using StatsBase
using Distributions
using Plots

function Least_squares_P(Y, P, Ω)
    r = size(P, 2)
    m = size(Y, 1)
    n = size(Y, 2)
    
    Q = Array{Float64, 2}(undef, n, r)

    for i = 1:n
        y = Y[:, i]
        p = Array{Float64, 2}(undef, m, r)

        for j = 1:r
            p[:, j] = Ω[:, i] .* P[:, j] 
        end

        Q'[:, i] = pinv(p) * y
    end

    return Q
end

function Least_squares_Q(Y, Q, Ω)
    r = size(Q, 2)
    m = size(Y, 1)
    n = size(Y, 2)

    P = Array{Float64, 2}(undef, m, r)

    for i = 1:m
        y = Y'[:, i]
        q = Array{Float64, 2}(undef, n, r)

        for j = 1:r
            q[:, j] = Ω'[:, i] .* Q[:, j] 
        end

        P'[:, i] = pinv(q) * y
    end

    return P
end

function Altmin(Y, P, Ω, iters)

    Q = Least_squares_P(Y, P, Ω)
    P = Least_squares_Q(Y, Q, Ω)
    
    for i = 1:iters-1
        Q = Least_squares_P(Y, P, Ω)
        P = Least_squares_Q(Y, Q, Ω)
    end

    return P, Q
end

function LRMCRec_AM(Y, Ω, r, iters)
    m = size(Y, 1)
    U, _, _ = svd(Y)
    P = U[:, 1:r]

    P_t, Q_t = Altmin(Y, P, Ω, iters)

    X = P_t * Q_t'

    return X
end


LRMCRec_AM (generic function with 1 method)

In [5]:
X  = LRMC_data_gen(32, 48, 8)
Y, Ω = Observation_samples(X, 32, 48, 196)

X_predict = LRMCRec_AM(Y, Ω, 8, 100)

32×48 Matrix{Float64}:
 -0.0416941    -0.187313     0.00642348  …  -0.562936     0.259352
 -0.00131982   -0.00268662   0.00414451     -0.0708499    0.00046994
 -0.00455454   -0.0186716   -0.00614353     -0.00212967   0.0337777
  0.0060761    -0.246116     0.0660126       0.252196    -0.162667
  0.0160429     0.039618    -0.00340605      0.0304046   -0.0078738
  0.0170226    -0.00751578   0.107588    …  -0.437306    -0.0118137
 -0.0806336     0.182507    -0.0367217       0.177678    -0.0937305
  0.0274882     0.164393    -0.0207649       0.546753    -0.0927862
  0.0766081     0.00646359   0.0483127       0.854538    -0.147075
  0.00522084   -0.21703      0.0511019       0.0410749   -0.0328091
  ⋮                                      ⋱               
 -0.0126735     0.171233    -0.0289607       0.0467297   -0.217098
 -0.00856316    0.25821     -0.0842809      -0.203098     0.104583
 -0.0362695     0.0390075    0.0183053   …   0.2999      -0.258449
 -0.0206781    -0.264268     0.0290492  

In [6]:
rank(X_predict)

8

### 3.2.2 Iterative Hard Thresholding (IHT) (20%)

Design, implement, and run simple tests for the IHT algorithm for low-rank matrix completion. Use the function name `LRMCRec_IHT`. Provide necessary documentation. 

In [10]:
function r_rank_approx(M, r::Int)
    U, Σ, V = svd(M)
    N = length(Σ)

    for i = r+1:N
        Σ[i] = 0
    end

    return U * Diagonal(Σ) * V'
end

function r_rank_approx_1(M, r::Int)
    # Gives a r rank approximation to the input matrix M 
    U, Sigma, V = svd(M)
    r_rank_M = Sigma[1]*U[:,1]*transpose(V[:,1])

    for i = 2:r
        u = U[:,i]; v = V[:,i]; s = Sigma[i]
        r_rank_M += s * u * transpose(v)
    end

    return r_rank_M
end 

function LRMCRec_IHT(Y, Ω, r::Int, τ::Float64, iters::Int)
    m, n = size(Y)
    X = copy(Y)
    
    temp = zeros(m,n)   
    error_list = []
    num_iters = iters

    while num_iters < 1000 
        # Iteration algorithm to reduce objective error 
        temp[Ω] = Y[Ω] - X[Ω]
        X  = r_rank_approx(X + τ*(temp), r)
        
        #Calculate objective error
        error = norm(Y[Ω] - X[Ω],2)^2
        append!(error_list,error)
        
        #Increase the number of iterations
        num_iters = num_iters + 1

    end 
    return X, 1:num_iters, error_list
end

LRMCRec_IHT (generic function with 1 method)

v = [0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.19599300006446388 -0.18106175009308564 0.0; 0.07427507620716461 0.0 0.0 0.0 0.0 0.0; 0.0 0.10218265144524336 0.0 -4.3519188267385524e-17 -4.020378476645396e-17 0.0]
p = 

[0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.19599300006446388 -0.18106175009308564 0.0; 0.07427507620716461 0.0 0.0 0.0 0.0 0.0; 0.0 0.10218265144524336 0.0 -4.3519188267385524e-17 -4.020378476645396e-17 0.0]


4×6 Matrix{Float64}:
 0.0        0.0       0.0   0.0           0.0          0.0
 0.0        0.0       0.0  -0.195993     -0.181062     0.0
 0.0742751  0.0       0.0   0.0           0.0          0.0
 0.0        0.102183  0.0  -4.35192e-17  -4.02038e-17  0.0

### 3.2.3 Iterative Shrinkage-Thresholding Algorithm (ISTA) (25%)

Design, implement, and run simple tests for ISTA (to solve the Lasso formulation) for low-rank matrix completion. Use the function name `LRMCRec_ISTA`. Provide necessary documentation. Use simulations to discuss the choice of parameters.

In [7]:
using Random
using LinearAlgebra
using StatsBase
using Distributions
using Plots

# Helper functions

function nuclear_norm(X)
    _, S, _ = svd(X)
    return sum(S)
end

function current_error(X, Y, Ω, λ)
    return 0.5 * norm(X[Ω] - Y[Ω])^2 + λ * nuclear_norm(X)
end

function Singular_value_soft_threshold(X, λ)
    U, S, V = svd(X)
    threshold = max.(S .- λ, 0.0)
    return U * Diagonal(threshold) * V'
end

# Low Rank Matrix Completion Iterative Shrinkage Thresholding Algorithm.

function LRMCRec_ISTA(Y, Ω, λ, iterations)
    X = copy(Y)

    error_list = zeros(iterations + 1)
    error_list[1] = current_error(X, Y, Ω, λ)

    for i=1:iterations
        X[Ω] = Y[Ω]
        X = Singular_value_soft_threshold(X, λ)

        error_list[i+1] = current_error(X, Y, Ω, λ)
    end

    return X, error_list
end

LRMCRec_ISTA (generic function with 1 method)

In [32]:
X  = LRMC_data_gen(32, 48, 8)
Y, Ω = Observation_samples(X, 32, 48, 384)

X_predict, error = LRMCRec_ISTA(Y, Ω, 0.01, 1000)

@show norm(X_predict - X) / norm(X)


norm(X_predict - X) / norm(X) = 0.5759336318892304


0.5759336318892304

### 3.2.4 Lasso-ADMM (25%)

Design, implement, and run simple tests for an ADMM algorithm (to solve the Lasso formulation) for low-rank matrix completion. Use the function name `LRMCRec_ADMM`. Provide necessary documentation. Compare ADMM and ISTA in terms of convergence.

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.